In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

import matplotlib.pyplot as plt
import numpy as np
import time
import open3d as o3d


TX_POSITION = [100, 170]
RX_GRID_SPACING = 2; 
RX_HEIGHT = 1.5
# CENTER_X, CENTER_Y, CENTER_Z = (27, 1, -3)
SCENE_NAME = "quarry"

RX_GRID_AREA = [50, 150, 120, 220]; # Meters x1, x2, y1, y2 set to [] to use whole area


# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass


# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

In [ ]:
o3dmesh = o3d.io.read_triangle_mesh(f"models/{SCENE_NAME}.ply")
o3dmesh = o3d.t.geometry.TriangleMesh.from_legacy(o3dmesh)
box = o3dmesh.get_axis_aligned_bounding_box()
z = box.max_bound[2].item() + 10 # 10 units above highest point is where rays start


if len(RX_GRID_AREA) == 4:
    min_x = RX_GRID_AREA[0]
    max_x = RX_GRID_AREA[1]
    min_y = RX_GRID_AREA[2]
    max_y = RX_GRID_AREA[3]
else:
    min_x = int(box.min_bound[0].item())
    max_x = int(box.max_bound[0].item())
    min_y = int(box.min_bound[1].item())
    max_y = int(box.max_bound[1].item())

scene = o3d.t.geometry.RaycastingScene()
scene.add_triangles(o3dmesh)
rays = []
ray_locations = []
num_elements_x = (int) ((max_x - min_x) / RX_GRID_SPACING) + 1
num_elements_y = (int) ((max_y - min_y) / RX_GRID_SPACING) + 1
print(num_elements_x * num_elements_y)
for col in range(num_elements_x):
    for row in range(num_elements_y):
        x = min_x + (col * RX_GRID_SPACING)
        y = min_y + (row * RX_GRID_SPACING)
        rays.append([x, y, z, 0, 0, -1])
        ray_locations.append((x, y))

rays = o3d.core.Tensor(rays, dtype=o3d.core.Dtype.Float32)

ans = scene.cast_rays(rays)

vertical_dist = ans['t_hit'].numpy()
points = []
for i, z_offset in enumerate(vertical_dist):
    if np.isfinite(z_offset): # check if ray hit
        x, y = ray_locations[i]
        points.append((x, y, z - z_offset + RX_HEIGHT))

tx_ray = o3d.core.Tensor([[TX_POSITION[0], TX_POSITION[1], z, 0, 0, -1]], dtype=o3d.core.Dtype.Float32)
tx_ans = scene.cast_rays(tx_ray)
tx_vertical_dist = tx_ans['t_hit'].numpy()
for i, z_offset in enumerate(tx_vertical_dist):
    if np.isfinite(z_offset): # check if ray hit
        tx_pos = (TX_POSITION[0], TX_POSITION[1], z - z_offset + RX_HEIGHT + .1)

In [ ]:
scene = load_scene(f"models/{SCENE_NAME}.xml")
scene.tx_array = PlanarArray(num_rows=8,
                          num_cols=2,
                          vertical_spacing=0.7,
                          horizontal_spacing=0.5,
                          pattern="tr38901",
                          polarization="VH")

# Configure antenna array for all receivers
scene.rx_array = PlanarArray(num_rows=1,
                          num_cols=1,
                          vertical_spacing=0.5,
                          horizontal_spacing=0.5,
                          pattern="dipole",
                          polarization="cross")

tx = Transmitter(name="tx",
              position=tx_pos,
              orientation=[0,0,0])
scene.add(tx)

# Create a receiver

for x, y, z in points:
    rx = Receiver(name=f"rx_{x}_{y}",
           position=[x, y, z],
           orientation=[0,0,0])
    scene.add(rx)

# TX points towards RX
tx.look_at(rx)

In [ ]:
scene.preview()